In [1]:
import gym 
import numpy as np

In [2]:
env_name = "CartPole-v1"
env = gym.make(env_name)
env.reset()

array([-0.01615046, -0.02847156, -0.04201839, -0.04516696])

In [ ]:
env.reset()   ####Test function

for _ in range(500):
    action = 1 ## = fordward force
    new_state, reward, done, _ = env.step(action) ## _ = info 
 ## print("something")
    env.render()
env.close()

In [ ]:
print("Observation space high: ", env.observation_space.high)
print("Observation space low", env.observation_space.low)
print("Actions: ", env.action_space.n)
print("-----------------------------------------------------------------")
print("Cart Possition: ",env.observation_space.high[0], env.observation_space.low[0])
print("Cart Velosity: ",env.observation_space.high[1], env.observation_space.low[1])
print("Pole Angle: ",env.observation_space.high[2], env.observation_space.low[2])
print("Pole Angular Velocity: ",env.observation_space.high[3], env.observation_space.low[3])

In [3]:
MAX_MIN_VELOCITY = 1.9

In [4]:
DISCRETE_OS_SIZE = [10,10,10,10]  ###  Size of state matrix

In [5]:
DISCRETE_OS_WIN_SIZE = [0,0,0,0]
DISCRETE_OS_WIN_SIZE[0] = (env.observation_space.high[0] - env.observation_space.low[0])/DISCRETE_OS_SIZE[0]
DISCRETE_OS_WIN_SIZE[1] = 2*MAX_MIN_VELOCITY/DISCRETE_OS_SIZE[1]
DISCRETE_OS_WIN_SIZE[2] = (env.observation_space.high[2] - env.observation_space.low[2])/DISCRETE_OS_SIZE[2]
DISCRETE_OS_WIN_SIZE[3] = 2*MAX_MIN_VELOCITY/DISCRETE_OS_SIZE[3]

In [18]:
q_table = np.random.uniform(low = 0, high = 1, size = (DISCRETE_OS_SIZE + [env.action_space.n]))
## Create a random Q table with values beetwen -2 and 0 in manifest size...

In [7]:
def get_discrete_state(state):    #### Convert the state
    if state[1] < MAX_MIN_VELOCITY*(-1):
        state[1] = -1*MAX_MIN_VELOCITY
    elif state[1] > MAX_MIN_VELOCITY:
        state[1] = MAX_MIN_VELOCITY
    if state[3] < MAX_MIN_VELOCITY*(-1):
        state[3] = -1*MAX_MIN_VELOCITY
    elif state[3] > MAX_MIN_VELOCITY:
        state[3] = MAX_MIN_VELOCITY
    env_low = env.observation_space.low
    env_low[1] = -1*MAX_MIN_VELOCITY
    env_low[3] = -1*MAX_MIN_VELOCITY
    discrete_state = (state - env_low) / DISCRETE_OS_WIN_SIZE
    if discrete_state[1] > DISCRETE_OS_SIZE[1] - 1:
        discrete_state[1] = DISCRETE_OS_SIZE[1] - 1
    if discrete_state[3] > DISCRETE_OS_SIZE[3] - 1:
        discrete_state[3] = DISCRETE_OS_SIZE[3] - 1
    return tuple(discrete_state.astype(np.int))

In [28]:
LEARNING_RATE = 0.1  ### a in equation 
DISCOUNT = 0.90  ### gamma, g, in equation
EPISODES = 15000
VIEW_EVERY = 1000

Q(s,a) = Q(s,a) + a x [ r(s,a) + g x argMaxQ(s,a)|a - Q(s,a) ]

In [42]:
env.reset()

max_score = 0

for episode in range(EPISODES):
    
    new_score = 0
    
    if episode % VIEW_EVERY == 0:
        view = True
    else:
        view = False
    
    done = False
    discrete_state = get_discrete_state(env.reset())

    while not done:
        action = np.argmax(q_table[discrete_state])
        new_state, reward, done, _ = env.step(action)
        new_discrete_state = get_discrete_state(new_state)
        if view:
            env.render()
        if not done:
            max_future_q = np.max(q_table[new_discrete_state])   #### Most important lines!!!!!!
            current_q = q_table[discrete_state + (action, )]
            new_q = (1 - LEARNING_RATE)*current_q + LEARNING_RATE*(reward + DISCOUNT * max_future_q)
            q_table[discrete_state + (action, )] = new_q
        new_score = new_score + 1
        if new_score > max_score:
            max_score = new_score

        discrete_state = new_discrete_state

env.close()

In [10]:
env.close()

In [30]:
print(max_score)

237


In [ ]:
261

In [49]:
from tensorflow import keras